In [ ]:
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn.metrics import confusion_matrix

import pandas as pd
import pickle as pk

cuda = torch.cuda.is_available()

In [ ]:
# Utils

def load_pickle(filename):
    try:
        p = open(filename, 'r')
    except IOError:
        print("Pickle file cannot be opened.")
        return None
    try:
        picklelicious = pk.load(p)
    except ValueError:
        print('load_pickle failed once, trying again')
        p.close()
        p = open(filename, 'r')
        picklelicious = pk.load(p)

    p.close()
    return picklelicious

def save_pickle(data_object, filename):
    pickle_file = open(filename, 'w')
    pk.dump(data_object, pickle_file)
    pickle_file.close()
    
def read_data(filename):
    print("Loading Data...")
    df = pd.read_csv(filename, header=None)
    data = df.values
    return data

def read_line(csvfile, line):
    with open(csvfile, 'r') as f:
        data = next(itertools.islice(csv.reader(f), line, None))
    return data

In [ ]:
#####################################################################################################################
#  Classifiers: Code directly modified based on the PyTorch Model Zoo Implementations
#####################################################################################################################

## 1D variant of VGG model take 200 dimensional fixed time series inputs
class VGG(nn.Module):

    def __init__(self, features, num_classes, arch='vgg'):
        super(VGG, self).__init__()
        self.arch = arch
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 6, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )
        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool1d(kernel_size=2, stride=2)]
        else:
            conv1d = nn.Conv1d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv1d, nn.BatchNorm1d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv1d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def vgg13bn(**kwargs):
    model = VGG(make_layers(cfg['B'], batch_norm=True), arch='vgg13bn', **kwargs)
    return model

def vgg16bn(**kwargs):
    model = VGG(make_layers(cfg['D'], batch_norm=True), arch='vgg16bn', **kwargs)
    return model

def vgg19bn(**kwargs):
    model = VGG(make_layers(cfg['E'], batch_norm=True), arch='vgg19bn', **kwargs)
    return model


## Multilayer LSTM based classifier taking in 200 dimensional fixed time series inputs
class LSTMClassifier(nn.Module):

    def __init__(self, in_dim, hidden_dim, num_layers, dropout, bidirectional, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.arch = 'lstm'
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
                input_size=in_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                dropout=dropout,
                bidirectional=bidirectional
            )

        self.hidden2label = nn.Sequential(
            nn.Linear(hidden_dim*self.num_dir, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, num_classes),
        )

        # self.hidden = self.init_hidden()

    def init_hidden(self):
        if cuda:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, x): # x is (batch_size, 1, 200), permute to (200, batch_size, 1)
        x = x.permute(2, 0, 1)
        # See: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426/2
        lstm_out, (h, c) = self.lstm(x, self.init_hidden())
        y  = self.hidden2label(lstm_out[-1])
        return y
    

## 1D Variant of ResNet taking in 200 dimensional fixed time series inputs
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, padding=1, stride=stride, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample

        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        # print('out', out.size(), 'res', residual.size(), self.downsample)
        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=1, padding=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=1, padding=1, stride=stride, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.conv3 = nn.Conv1d(planes, planes * 4, kernel_size=1, padding=1, stride=stride, bias=False)
        self.bn3 = nn.BatchNorm1d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, arch):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1]) #, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2]) #, stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3]) #, stride=2)
        self.avgpool = nn.AvgPool1d(7, stride=1)
        self.fc = nn.Linear(22528 , num_classes) # 512 * block.expansion
        
        self.arch = arch

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.fc(x)

        return x

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], arch='resnet18', **kwargs)

    return model

def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], arch='resnet34', **kwargs)

    return model

def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], arch='resnet50', **kwargs)

    return model

def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], arch='resnet101', **kwargs)

    return model

def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], arch='resnet152', **kwargs)
    
    return model

In [ ]:
#####################################################################################################################
#  DataLoaders
#####################################################################################################################

# Dummy Dataset for testing purpose only
class DummyDataset(Dataset):
    """Time Series dataset."""

    def __init__(self, numclasses=15):
        self.numclasses = numclasses

    def __len__(self):
        return 512

    def __getitem__(self, idx):
        data = np.random.randn(200)
        data = np.expand_dims(data, axis=0)

        return data, np.random.randint(self.numclasses)
    
# Synchronized Time Series, allow variable training window
class TimeSeriesDataset(Dataset):
    """Time Series dataset."""

    def __init__(self, data_path, norm, is_train, train_window):

        self.file_dirs = sorted([f for f in glob.glob('%s/*/*/*'%data_path) if os.path.isdir(f)])
        self.norm = norm
        self.is_train = is_train
        self.train_window = train_window
        self.files_per_dir = self.train_window**2 if self.is_train else (32**2 - self.train_window**2)

    def __len__(self):
        return int( len(self.file_dirs) * self.files_per_dir )

    def __getitem__(self, idx):
        fdir = self.file_dirs[ idx // self.files_per_dir ] 
        file_id = idx % self.files_per_dir + ( 0 if self.is_train else (self.train_window**2) )
        data = np.fromfile(os.path.join(fdir, '%s.bin'%str(file_id).zfill(4)))
        target = int(fdir.split('/')[3]) # Be careful about this index !!
        
        if self.norm:
            data -= data.mean()
            data /= data.std()
        else:
            data -= data[0]
        
        data = np.expand_dims(data, axis=0)
        return data, target

In [ ]:
#####################################################################################################################
#  Trainer and Core Experiment Scripts
#####################################################################################################################

def run_trainer(experiment_path, model_path, model, train_loader, test_loader, get_acc, resume, batch_size, num_epoch):

    if not os.path.exists(model_path):
        os.makedirs(model_path)
    def save_checkpoint(state, is_best, filename=model_path+'checkpoint.pth.tar'):
        torch.save(state, filename)
        if is_best:
            shutil.copyfile(filename, model_path+'model_best.pth.tar')
    def get_last_checkpoint(model_path):
        fs = sorted([f for f in os.listdir(model_path) if 'Epoch' in f], key=lambda k: int(k.split()[1]))
        return model_path+fs[-1] if len(fs) > 0 else None
    
    start_epoch = 0
    best_res = 0
    lrcurve = []
    conf_mats = []
    resume_state = get_last_checkpoint(model_path) if resume else None
    if resume_state and os.path.isfile(resume_state):
        print("=> loading checkpoint '{}'".format(resume_state))
        checkpoint = torch.load(resume_state)
        start_epoch = checkpoint['epoch']+1
        best_res = checkpoint['val_acc']
        lrcurve = checkpoint['lrcurve']
        conf_mats = checkpoint['conf_mats']
        model.load_state_dict(checkpoint['state_dict'])
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters())
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resume_state, checkpoint['epoch']))
    else:
        if cuda:
            model.cuda()
        optimizer = optim.Adam(model.parameters())

    criterion = nn.CrossEntropyLoss()
    # scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5) # optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5)

    def train(epoch):
        model.train()
        total, total_correct = 0., 0.
        train_conf_mats = []
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data.float()), Variable(target.long())
            if cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            _, correct, num_instance, conf_mat = get_acc(output, target)
            total_correct += correct
            total += num_instance
            train_conf_mats.append(conf_mat)
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Acc: {:.2f}%/{:.2f}%'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0],
                    100. * correct / num_instance, 100. * total_correct / total ))
        
        return 100. * total_correct / total, np.dstack(train_conf_mats).sum(axis=2)

    def test():
        model.eval()
        test_loss = 0.
        total, total_correct = 0., 0.
        test_conf_mats = []
        preds = []
        for data, target in test_loader:
            data, target = Variable(data.float(), volatile=True), Variable(target.long())
            if cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).data[0] # sum up batch loss
            
            pred, correct, num_instance, conf_mat = get_acc(output, target)
            total_correct += correct
            total += num_instance
            test_conf_mats.append(conf_mat)
            preds.append(pred)

        test_acc = 100. * total_correct / total
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, total_correct, total,
            test_acc))

        return np.vstack(preds), test_acc, np.dstack(test_conf_mats).sum(axis=2)


    for epoch in range(start_epoch, num_epoch):
        is_best = False

        train_acc, train_conf = train(epoch)
        preds, val_acc, val_conf = test()
        
        print("Training Confmat: ")
        print(train_conf)
        print("Testing Confmat: ")
        print(val_conf)
        print("Number of Predictions Made: ")
        print(preds.shape)
        
        lrcurve.append((train_acc, val_acc))
        conf_mats.append((train_conf, val_conf))
        # scheduler.step(val_loss)

        if val_acc > best_res:
            best_res = val_acc
            is_best = True

        save_checkpoint({
                'epoch': epoch,
                'arch': model.arch,
                'state_dict': model.cpu().state_dict(),
                'train_acc':train_acc,
                'val_acc': val_acc,
                'optimizer' : optimizer.state_dict(),
                'lrcurve':lrcurve,
                'train_conf':train_conf,
                'val_conf':val_conf,
                'conf_mats':conf_mats,
                'test_predictions':preds,
            }, is_best,
            model_path+"Epoch %d Acc %.4f.pt"%(epoch, val_acc))

        if cuda:
            model.cuda()
            
    return lrcurve, conf_mats

def run_dummy_experiment(experiment_path, model_root, models, norm, train_window, get_acc, resume=False, num_epoch=10):
    
    exp_result = {}
    for batch_size, model in models:
        print("Running %s" % model.arch)
        
        print('Loading Data..')
        train_data = DummyDataset()
        test_data = DummyDataset()
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=16)
    
        model_path = os.path.join(
            experiment_path, 
            model_root,
            'norm' if norm else 'nonorm',
            str(train_window),
            model.arch) + '/'
        lrcurve, conf_mats = run_trainer(experiment_path, model_path, model, train_loader, test_loader, get_acc, resume, batch_size, num_epoch)
        exp_result[model.arch] = {'lrcurve':lrcurve, 'conf_mats':conf_mats}
        
    return exp_result
        
def run_experiment(experiment_path, data_path, model_root, models, norm, train_window, get_acc, resume=False, num_epoch=10):
    
    exp_result = {}
    for batch_size, model in models:
        print("Running %s" % model.arch)
        
        print('Loading Data..')
        train_data = TimeSeriesDataset(data_path, norm, True, train_window)
        test_data = TimeSeriesDataset(data_path, norm, False, train_window)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)
    
        model_path = os.path.join(
            experiment_path, 
            model_root,
            'norm' if norm else 'nonorm',
            str(train_window),
            model.arch) + '/'
        lrcurve, conf_mats = run_trainer(experiment_path, model_path, model, train_loader, test_loader, get_acc, resume, batch_size, num_epoch)
        exp_result[model.arch] = {'lrcurve':lrcurve, 'conf_mats':conf_mats}
        
    return exp_result

In [ ]:
#####################################################################################################################
#  Dummy Experiment for Testing Purpose
#####################################################################################################################

def get_models():
    return [
        (1024, vgg13bn(num_classes=15)),
        (256, LSTMClassifier(
            in_dim=1,
            hidden_dim=120,
            num_layers=3,
            dropout=0.8,
            bidirectional=True,
            num_classes=15,
            batch_size=256
        )),
        (1024, resnet34(num_classes=15))
    ]

def get_acc(output, target):
    # takes in two tensors to compute accuracy
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.data.view_as(pred)).cpu().sum()
    conf_mat = confusion_matrix(pred.cpu().numpy(), target.data.cpu().numpy(), labels=range(15))
    return pred.cpu().numpy(), correct, target.size(0), conf_mat

dummy_experiment = {
    'experiment_path':'dummy', 
    'model_root':'model', 
    'models':get_models(), 
    'norm':False, 
    'train_window':24, 
    'get_acc': get_acc,
    'resume':False, 
    'num_epoch':3
}

# exp_log = run_dummy_experiment(**experiment)
# print(exp_log)
# experiment_filepath = os.path.join(
#                         experiment['experiment_path'], 
#                         experiment['model_root'],
#                         'norm' if experiment['norm'] else 'nonorm',
#                         str(experiment['train_window']),
#                         'exp_log.pkl')
# save_pickle(exp_log, experiment_filepath)

In [ ]:
#####################################################################################################################
#  Material Recognition Experiment with Deep Models
#####################################################################################################################

def get_models(): # tuples of (batch_size, model)
    return [
        (1024, vgg13bn(num_classes=15)),
        (256, LSTMClassifier(
            in_dim=1,
            hidden_dim=120,
            num_layers=3,
            dropout=0.8,
            bidirectional=True,
            num_classes=15,
            batch_size=256
        )),
        (1024, resnet34(num_classes=15))
    ]

def get_acc(output, target):
    # takes in two tensors to compute accuracy
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.data.view_as(pred)).cpu().sum()
    conf_mat = confusion_matrix(pred.cpu().numpy(), target.data.cpu().numpy(), labels=range(15))
    return np.squeeze(pred.cpu().numpy()), correct, target.size(0), conf_mat


experiments = [
    {
        'experiment_path':'roll', 
        'data_path':'roll/roll',
        'model_root':'model', 
        'models':get_models(),
        'norm':False, 
        'train_window':24, 
        'get_acc': get_acc,
        'resume':True,  
        'num_epoch':10
    },
    
    {
        'experiment_path':'roll', 
        'data_path':'roll/roll',
        'model_root':'model', 
        'models':get_models(),
        'norm':True, 
        'train_window':24, 
        'get_acc': get_acc,
        'resume':True, 
        'num_epoch':10
    },
]

for experiment in experiments:
    exp_log = run_experiment(**experiment)
    print(exp_log)
    experiment_filepath = os.path.join(
                        experiment['experiment_path'], 
                        experiment['model_root'],
                        'norm' if experiment['norm'] else 'nonorm',
                        str(experiment['train_window']),
                        'exp_log.pkl')
    save_pickle(exp_log, experiment_filepath)